# Pipeline


In [3]:
import pandas as pd
import requests
import numpy as np
import missingno as msno
import math
import matplotlib.pyplot as plt
import json
import pickle
import seaborn as sns

In [ ]:
def cambiar_minusculas_variable(archivo,variable):
    archivo[variable]=archivo[variable].str.lower()
    


def tabla_estadisticos_descriptivos_variables_numericas(archivo):
    def selecciona_variables_numericas(archivo):
        lista_variables_numericas=archivo.select_dtypes(include = 'number').columns.values
        archivo_variables_numericas = archivo.loc[:, lista_variables_numericas]
        return archivo_variables_numericas
    def q25(archivo):
        qal25=archivo.quantile(.25)
        return qal25
    def q75(archivo):
        qal75=archivo.quantile(.75)
        return qal75
    def uniques(archivo):
        uni=archivo.nunique()
        return uni
    def missings(archivo):
        nulos=archivo.isnull().sum()
        return nulos
    def prop_missings(archivo):
        prop=(archivo.isnull().sum()/len(archivo.index))*100
        return prop
    archivonum=selecciona_variables_numericas(archivo).agg(['max','min','mean','std',q25,'median',q75,'skew','kurt','count',uniques,missings,prop_missings])
    return archivonum

def tabla_estadisticos_descriptivos_variables_categoricas(archivo):
    def selecciona_variables_categoricas(archivo):
        lista_variables_categoricas=archivo.select_dtypes(include = 'object').columns.values
        archivo_variables_categoricas = archivo.loc[:, lista_variables_categoricas]
        return archivo_variables_categoricas
    def uniques(archivo):
        uni=archivo.nunique()
        return uni
    def uniques_list(archivo):
        uni=archivo.unique()
        return uni
    def missings(archivo):
        nulos=archivo.isnull().sum()
        return nulos
    def prop_missings(archivo):
        prop=(archivo.isnull().sum()/len(archivo.index))*100
        return prop
    archivocat=selecciona_variables_categoricas(archivo).agg(['count',uniques,uniques_list,missings,prop_missings])
    return archivocat

def grafico_histograma_categorica(archivo,variable_categorica,variable_contar,etiqueta_ejex,etiqueta_ejey,titulo):
    result=archivo.groupby([variable_categorica])[variable_contar].count().reset_index().sort_values(variable_contar,ascending=False)
    graf=sns.countplot(x=variable_categorica, palette="Paired", data=archivo,order=result[variable_categorica].values);
    graf.set_xticklabels(graf.get_xticklabels(), rotation=45, ha="right")
    graf.set(xlabel=etiqueta_ejex, ylabel=etiqueta_ejey, title=titulo)    

def grafico_barplot_orden_decreciente(archivo,variable_ejex,variable_ejey,etiqueta_ejex,etiqueta_ejey,titulo):
    result = archivo.groupby([variable_ejex])[variable_ejey].mean().reset_index().sort_values(variable_ejey,ascending=False)
    graf=sns.barplot(x=variable_ejex, y=variable_ejey, data=archivo, order=result[variable_ejex])
    graf.set_xticklabels(graf.get_xticklabels(), rotation=40, ha="right")
    graf.set(xlabel=etiqueta_ejex, ylabel=etiqueta_ejey, title=titulo)



In [4]:
data = pd.read_csv('../data/201912-citibike-tripdata.csv.zip')

In [5]:
#Creación de la columna trip_id

#generamos los ID's
n= len(data.index)
digits_range = range(1345,n + 1345)
digits_list = list(digits_range)

#Agreagamos la columna al df
data['trip_id'] = digits_list

In [6]:
#Se obtienen las filas que tienen como usertype "Customer"
subset_c = data.loc[data['usertype'] == 'Customer']

#Se obtienen las filas que tienen como usertype "Subscriber"
subset_s = data.loc[data['usertype'] == 'Subscriber']

In [7]:
subset_c.shape[0]

#Número de pases de 1 día
one_day = round(subset_c.shape[0]*.915)

#Número de pases de 3 días
tree_day = subset_c.shape[0] - one_day

print("Numero de pasese en diciembre de 1 día",one_day )
print("Numero de pasese en diciembre de 3 días",tree_day )
one_day + tree_day

Numero de pasese en diciembre de 1 día 71027
Numero de pasese en diciembre de 3 días 6598


77625

In [8]:
#Creación de columna auxiliar para separar los pases de Costumer en 2
subset_c['aux']=np.random.binomial(1,.915,77625)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
#Creación de columna con el nombre de los pases nuevos
subset_c['type_pass'] = np.where(subset_c['aux'] == 1, "one-day", "tree-days")
#Borramos la columa auxiliar
del subset_c['aux']

subset_s['type_pass']='annual'
#Juntamos los dos dataframes
data = subset_s.append(subset_c, ignore_index=True)
data.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(955210, 17)

In [11]:
#Creación de una columna nueva : trip_category
data['aux']=np.random.binomial(1,0.0454,955210)

data['trip_category'] = np.where(data['aux'] == 1, "one-way", "round-trip")
#Borramos la columa auxiliar
del data['aux']

In [12]:
data['duration'] = round(data['tripduration']/60)
data['starttime'] = pd.to_datetime(data['starttime']).dt.strftime('%Y-%m-%d %H:%M')
data['stoptime'] = pd.to_datetime(data['stoptime']).dt.strftime('%Y-%m-%d %H:%M')

In [13]:
data_ = data.copy()

In [14]:
# remove spaces in column names
data_.columns = data_.columns.str.replace(' ', '_')

In [15]:
# Change to lower categorical variables
cambiar_minusculas_variable(data_,'start_station_name')
cambiar_minusculas_variable(data_,'end_station_name')
cambiar_minusculas_variable(data_,'usertype')
cambiar_minusculas_variable(data_,'type_pass')
cambiar_minusculas_variable(data_,'trip_category')

In [16]:
# Converting start and end times to datetime objects.
data_['starttime'] = pd.to_datetime(data_['starttime'])
data_['stoptime'] = pd.to_datetime(data_['stoptime'])

In [17]:
# Create label for rides that were charged based on current pricing plan.
def label_charged(duration):
    """"Function to create a new catergorical variable that indicate if theres has to be an adittional charge"""
    if duration <= 30:
        return 0
    else:
        return 1

data_['charged_ride'] = data_['duration'].apply(label_charged)

In [18]:
# Column for hour of day of ride.
data_['start_hour'] = data_['starttime'].dt.hour

In [19]:
latest_ride_date = data_['stoptime'].max()
latest_ride_date

Timestamp('2020-01-17 20:00:00')

In [20]:
bike_last_used_id = bike_last_used = data_.groupby('bikeid')['stoptime'].max().keys().tolist()
bike_last_used_date = data_.groupby('bikeid')['stoptime'].max().tolist()

In [21]:
# Number of bikes active each month in the past year.
for month in range(1,13):
    print(month, len([record for record in list(zip(bike_last_used_id,bike_last_used_date)) if 
                      record[1] > latest_ride_date + np.timedelta64(-month, 'M') ]))

1 12454
2 13816
3 13816
4 13816
5 13816
6 13816
7 13816
8 13816
9 13816
10 13816
11 13816
12 13816


In [22]:
# Number of inactive bikes (bikes that haven't been used in the past 6 months)
len([record for record in list(zip(bike_last_used_id,bike_last_used_date)) if 
     record[1] < latest_ride_date + np.timedelta64(-6, 'M') ])

0

In [23]:
# Create feature indicating active bikes, used in past 6 months. 1 if active, 0 if inactive.
inactive_ids = [record[0] for record in list(zip(bike_last_used_id,bike_last_used_date)) if 
                record[1] < latest_ride_date + np.timedelta64(-6, 'M') ]
data_['active_6m'] = data_['bikeid'].apply(lambda bike_id: 0 if bike_id in (inactive_ids) else 1)

In [25]:
#Select variables to modeling
data_ = data_ [["trip_id", "duration","starttime", "stoptime", "trip_category", "type_pass"]]

In [26]:
data_

,trip_id,duration,starttime,stoptime,trip_category,type_pass
0,1345,10.0,2019-12-01 00:00:00,2019-12-01 00:10:00,round-trip,annual
1,1347,12.0,2019-12-01 00:00:00,2019-12-01 00:12:00,round-trip,annual
2,1349,18.0,2019-12-01 00:00:00,2019-12-01 00:17:00,round-trip,annual
3,1350,21.0,2019-12-01 00:00:00,2019-12-01 00:21:00,round-trip,annual
4,1351,10.0,2019-12-01 00:00:00,2019-12-01 00:10:00,round-trip,annual
...,...,...,...,...,...,...
955205,956539,8.0,2019-12-31 23:53:00,2020-01-01 00:00:00,round-trip,one-day
955206,956542,8.0,2019-12-31 23:53:00,2020-01-01 00:01:00,round-trip,one-day
955207,956544,30.0,2019-12-31 23:54:00,2020-01-01 00:24:00,round-trip,one-day
955208,956547,30.0,2019-12-31 23:55:00,2020-01-01 00:24:00,round-trip,one-day
